In [6]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, mean_squared_error, r2_score)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

# ==================== 参数配置 ====================
input_path = "预测数据归一化后-20251120.xlsx"  # 输入数据路径
output_dir = "xgboost_model_p"     # 输出目录
target_column = "SO2 tolerance"     # 目标变量列名
outer_test_size = 0.2               # 外层测试集比例
inner_test_size=0.2                 # 内层测试集比例   
random_state = 42                   # 随机种子
n_splits = 5                        # 交叉验证折数
model_path = os.path.join(output_dir, "xgboost_model.pkl")  # 模型保存路径

# ==================== 函数定义 ====================
def prepare_data(input_path, target_column):
    """数据加载与预处理"""
    df = pd.read_excel(input_path)
    if target_column not in df.columns:
        raise ValueError(f"目标列 {target_column} 不存在于数据中")
    
    y = df[target_column]
    X = df.drop(columns=[target_column])
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
    return X[numeric_cols], y

def determine_problem_type(y):
    """自动判断问题类型"""
    if y.nunique() <= 10:  # 分类问题（类别数≤10）
        return ('binary:logistic' if y.nunique() == 2 else 'multi:softprob', 
                "classification")
    return ('reg:squarederror', "regression")

def get_model_params(objective, random_state):
    """获取模型参数"""
    return {
        'objective': objective,
        'eval_metric': 'logloss' if objective.startswith('binary') else 'rmse',
        'eta': 0.1,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'seed': random_state,
        'n_jobs': -1
    }

def train_and_evaluate(X_train, y_train, X_test, y_test, params, problem_type):
    """训练模型并返回评估结果"""
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dtest, 'test')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # 评估模型
    ddata_train = xgb.DMatrix(X_train)
    ddata_test = xgb.DMatrix(X_test)
    
    if problem_type == "classification":
        y_pred_train =np.round(model.predict(ddata_train)) if params['objective'] == 'binary:logistic' else np.argmax(model.predict(ddata_train), axis=1)
        y_pred_test =np.round(model.predict(ddata_test)) if params['objective'] == 'binary:logistic' else np.argmax(model.predict(ddata_test), axis=1)
        metrics = {
            'Train Accuracy': accuracy_score(y_train, y_pred_train),
            'Test Accuracy': accuracy_score(y_test, y_pred_test),
            'Train F1': f1_score(y_train, y_pred_train, average='macro'),
            'Test F1': f1_score(y_test, y_pred_test, average='macro')
        }
    else:
        y_pred_train = model.predict(ddata_train)
        y_pred_test = model.predict(ddata_test)
        metrics = {
            'Train R2': r2_score(y_train, y_pred_train),
            'Test R2': r2_score(y_test, y_pred_test),
            'Train RMSE': np.sqrt(mean_squared_error(y_train, y_pred_train)),
            'Test RMSE': np.sqrt(mean_squared_error(y_test, y_pred_test))
        }
    return model, metrics
def plot_feature_importance(model, output_dir):
    """
    绘制特征重要性图（使用自定义颜色方案）
    
    参数:
        model: 训练好的XGBoost模型
        output_dir: 输出目录
    """
    # 自定义颜色方案（根据用户提供的颜色）
    custom_colors = {
        'EN.': '#82BBF0', 
        'P.Tim.': '#DFF1FF', 
        'S.A.': '#FDB3CC', 
        'I.R': '#82BBF0', 
        'I.E.': '#82BBF0', 
        'P.D.': '#FDB3CC',
        'H2O': '#FF78A8', 
        'P.Tem.': '#DFF1FF', 
        'M.P.': '#82BBF0', 
        'T.C.': '#82BBF0', 
        'P.V.': '#FDB3CC', 
        'C.Tem.': '#FDB3CC',
        'Density': '#82BBF0',
        'NH3': '#FF78A8', 
        'SO2': '#DFF1FF', 
        'C.Tim.': '#FDB3CC', 
        'O2': '#FF78A8', 
        'NO': '#FF78A8',
        'GHSV': '#FF78A8'
    }
    
    # 设置全局字体样式
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.weight'] = 'bold'
    
    # 设置图形大小和边缘距离
    fig, ax = plt.subplots(figsize=(10, 12), facecolor='none')
    plt.subplots_adjust(left=0.2, right=0.95, top=0.9, bottom=0.1)
    ax.grid(False)
    ax.set_facecolor('none')  # 透明背景
    
    
    # 获取特征重要性数据
    importance = model.get_score(importance_type='weight')
    importance = sorted(importance.items(), key=lambda x: x[1], reverse=False)
    feature_names = [x[0] for x in importance]
    feature_imp = [x[1] for x in importance]
    
    # 为每个特征分配颜色（如果特征未在自定义颜色中定义，使用默认颜色）
    colors = [custom_colors.get(feature, '#FF78A8') for feature in feature_names]
    
    # 绘制水平条形图
    bars = ax.barh(
        range(len(feature_names)), 
        feature_imp, 
        height=0.8,  # 调整柱形宽度
        color=colors,  # 使用自定义颜色
        
        
    )
    
    # 设置y轴刻度
    ax.set_yticks(range(len(feature_names)))
    ax.set_yticklabels(feature_names)
    
    # 设置x轴标签
    plt.xlabel(
        "Importance",
        fontdict={
            'fontname': 'Arial',
            'fontsize': 24,
            'fontweight': 'bold'
        },
        labelpad=10  # 标签与轴的距离
    )
    
    # 设置y轴标签
    plt.ylabel(
        ' ',
        fontdict={
            'fontname': 'Arial',
            'fontsize': 20,
            'fontweight': 'bold'
        },
        labelpad=10  # 标签与轴的距离
    )
    
    # 设置坐标轴刻度字体
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontname('Arial')
        label.set_fontsize(20)
    
    # 保留所有边框并设置样式
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(2.5)  # 加粗边框线
        spine.set_color('black')  # 设置边框颜色
    
    # 添加数值标签
    for bar in bars:
        width = bar.get_width()
        ax.text(
            width + max(feature_imp)*0.01,  # 位置微调
            bar.get_y() + bar.get_height()/2,
            f'{width:.1f}',
            ha='left',
            va='center',
            fontname='Arial',
            fontsize=18,
            color='none' 
        )
    
    # 保存图像
    plt.savefig(
        os.path.join(output_dir, "feature_importance.png"), 
        dpi=300, 
        bbox_inches='tight',
        transparent=False
    )
    plt.close()
    
def plot_actual_vs_predicted(X_train, y_train, X_test, y_test, model, output_dir):
    """绘制实际值 vs 预测值图（回归问题）"""
    # 设置全局字体样式
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.weight'] = 'bold'
        
    d_train = xgb.DMatrix(X_train)
    pp_tr = model.predict(d_train)
    d_test = xgb.DMatrix(X_test)
    y_predicted = model.predict(d_test)
    
    plt.figure(figsize=(10, 12))
    g = sns.JointGrid(x=y_test, y=y_predicted, height=8, space=0)
    
    # 绘制散点图和回归线
    sns.scatterplot(x=y_train, y=pp_tr, s=100, color='#DB7987', ax=g.ax_joint, label='Training')
    sns.scatterplot(x=y_test, y=y_predicted, s=100, color='#82BBF0', ax=g.ax_joint, label='Testing')
    sns.regplot(x=y_test, y=y_predicted, ax=g.ax_joint, scatter=False, color='gray')
       
    # 调整坐标轴刻度标签样式
    for ax in [g.ax_joint, g.ax_marg_x, g.ax_marg_y]:
        ax.tick_params(
            axis='both', 
            which='major', 
            labelsize=20,      # 刻度标签大小
            colors='black',   # 刻度标签颜色
            width=3,
            length=8,
            direction='out'

        )
    # 设置图例样式
    g.ax_joint.legend(
        title_fontsize=20,      # 标题字体大小
        fontsize=20,           # 项目字体大小
        frameon=False,          # 显示边框
        framealpha=1,        # 边框透明度
        edgecolor='white',   # 边框颜色
        facecolor='white'      # 背景颜色
    )
    
    # 主图边框（闭合，线宽3）
    for spine in g.ax_joint.spines.values():
        spine.set_visible(True)
        spine.set_edgecolor('#333333')
        spine.set_linewidth(2)
    
    # 计算实际值和预测值的全局范围
    all_values = np.concatenate([y_train, y_test, pp_tr, y_predicted])
    min_val = np.min(all_values)
    max_val = np.max(all_values)
    buffer = (max_val - min_val) * 0.15  # 增加缓冲区，确保边缘数据有足够空间
    
    # 设置主图坐标范围
    g.ax_joint.set_xlim(min_val - buffer, max_val + buffer)
    g.ax_joint.set_ylim(min_val - buffer, max_val + buffer)
    
    # 绘制理想预测线
    g.ax_joint.plot([min_val, max_val], [min_val, max_val], '--', color='gray', alpha=0.5)
    
    # 优化边缘分布的面积图
    # X轴面积图 - 使用更合适的带宽和范围
    sns.kdeplot(
        x=y_train, ax=g.ax_marg_x, color='#DB7987', alpha=0.5,
        fill=True, linewidth=0, label='Training', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    sns.kdeplot(
        x=y_test, ax=g.ax_marg_x, color='#82BBF0', alpha=0.5,
        fill=True, linewidth=0, label='Testing', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    
    # y轴面积图 - 使用更合适的带宽和范围
    sns.kdeplot(
        y=pp_tr, ax=g.ax_marg_y, color='#DB7987', alpha=0.5,
        fill=True, linewidth=0, label='Training', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    sns.kdeplot(
        y=y_predicted, ax=g.ax_marg_y, color='#82BBF0', alpha=0.5,
        fill=True, linewidth=0, label='Testing', bw_adjust=1.5,
        clip=(min_val - buffer, max_val + buffer)
    )
    
    # 隐藏边缘分布的图例
    legend_x = g.ax_marg_x.legend()
    legend_x.set_visible(False)
    legend_y = g.ax_marg_y.legend()
    legend_y.set_visible(False)
    
    # 调整边缘分布坐标轴粗细和刻度方向
    for ax in [g.ax_marg_x, g.ax_marg_y]:
        for spine in ax.spines.values():
            spine.set_linewidth(2)
    
    # 新增：设置边缘坐标轴刻度线朝外
        ax.tick_params(
           axis='both', 
           which='major',
           direction='out',  # 使刻度线朝外
           width=2,          # 刻度线粗细与轴线一致
           length=8          # 刻度线长度
    )
    
    # 确保边缘分布与主图坐标范围一致
    g.ax_marg_x.set_xlim(g.ax_joint.get_xlim())
    g.ax_marg_y.set_ylim(g.ax_joint.get_ylim())
    
    # 关键修改：在所有绘图操作完成后设置轴标签
    g.ax_joint.set_xlabel("Actual value", fontsize=26, fontname='Arial', fontweight='bold')
    g.ax_joint.set_ylabel("Predicted value", fontsize=26, fontname='Arial', fontweight='bold')
    
    # 调整整体布局，确保有足够空间显示面积图
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.12, right=0.95)
    
    # 保存图像
    plot_path = os.path.join(output_dir, "actual_vs_predicted.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    return plot_path

# ==================== 主程序 ====================
def main():
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. 数据准备
    X, y = prepare_data(input_path, target_column)
    objective, problem_type = determine_problem_type(y)
    params = get_model_params(objective, random_state)
    
    # 2. 外层数据划分
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=outer_test_size, random_state=random_state)
    
    # 3. 外层模型训练与评估
    print("外层模型训练与评估...")
    outer_model, outer_metrics = train_and_evaluate(
        X_train, y_train, X_test, y_test, params, problem_type)
    joblib.dump(outer_model, model_path)
    print(f"\n外层模型已保存至：{model_path}")
    
    # 打印外层评估结果
    print("\n外层模型性能：")
    print(pd.DataFrame([outer_metrics], index=['Value']))
    
    # 4. 内层5折交叉验证
    print(f"\n开始内层{n_splits}折交叉验证...")
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    cv_metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
        print(f"\n正在处理第 {fold} 折...")
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model, metrics = train_and_evaluate(X_tr, y_tr, X_val, y_val, params, problem_type)
        cv_metrics.append(metrics)
        print(f"第 {fold} 折完成 - 验证集R2: {metrics.get('Test R2', metrics.get('Test Accuracy')):.4f}")
    
    # 5. 交叉验证结果分析
    cv_results = pd.DataFrame(cv_metrics)
    print("\n交叉验证结果汇总：")
    print(cv_results)
    
    mean_metrics = cv_results.mean().to_dict()
    std_metrics = cv_results.std().to_dict()
    
    print("\n交叉验证平均性能：")
    for metric in mean_metrics:
        print(f"{metric}: {mean_metrics[metric]:.4f} ± {std_metrics[metric]:.4f}")
    
    # 6. 可视化与结果保存
    plot_feature_importance(outer_model, output_dir)
    
    if problem_type == "regression":
        plot_path = plot_actual_vs_predicted(X_train, y_train, X_test, y_test, 
                                           outer_model, output_dir)
        print(f"\n实际值 vs 预测值图已保存至：{plot_path}")
    
    # 7. 保存评估结果
    results = {
        'outer_test_r2': outer_metrics.get('Test R2', outer_metrics.get('Test Accuracy')),
        'cv_mean_r2': mean_metrics.get('Test R2', mean_metrics.get('Test Accuracy')),
        'cv_std_r2': std_metrics.get('Test R2', std_metrics.get('Test Accuracy'))
    }
    
    with open(os.path.join(output_dir, "results_summary.txt"), 'w') as f:
        f.write("模型评估结果汇总:\n")
        f.write(f"外层测试集R2: {results['outer_test_r2']:.4f}\n")
        f.write(f"交叉验证平均R2: {results['cv_mean_r2']:.4f} ± {results['cv_std_r2']:.4f}\n")
    
    print("\n评估结果已保存至 results_summary.txt")
    print("\n嵌套交叉验证完成！所有结果保存在目录：", output_dir)

if __name__ == "__main__":
    main()   


外层模型训练与评估...

外层模型已保存至：xgboost_model_p\xgboost_model.pkl

外层模型性能：
       Train R2   Test R2  Train RMSE  Test RMSE
Value  0.999906  0.798274     0.00093   0.053508

开始内层5折交叉验证...

正在处理第 1 折...
第 1 折完成 - 验证集R2: 0.6247

正在处理第 2 折...
第 2 折完成 - 验证集R2: 0.2189

正在处理第 3 折...
第 3 折完成 - 验证集R2: 0.3950

正在处理第 4 折...
第 4 折完成 - 验证集R2: 0.1628

正在处理第 5 折...
第 5 折完成 - 验证集R2: 0.1160

交叉验证结果汇总：
   Train R2   Test R2  Train RMSE  Test RMSE
0  0.999811  0.624740    0.001389   0.044537
1  0.987335  0.218892    0.010806   0.081226
2  0.993186  0.394989    0.008324   0.056880
3  0.993064  0.162828    0.006520   0.131639
4  0.975746  0.115967    0.015749   0.066475

交叉验证平均性能：
Train R2: 0.9898 ± 0.0090
Test R2: 0.3035 ± 0.2084
Train RMSE: 0.0086 ± 0.0053
Test RMSE: 0.0762 ± 0.0338

实际值 vs 预测值图已保存至：xgboost_model_p\actual_vs_predicted.png

评估结果已保存至 results_summary.txt

嵌套交叉验证完成！所有结果保存在目录： xgboost_model_p


In [19]:
# -*- coding: utf-8 -*-
import shap
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from matplotlib import rcParams

# ==================== 参数配置 ====================
model_path = "xgboost_model_p/xgboost_model.pkl"  # 模型路径
input_path = "预测数据归一化后-20251120.xlsx"                # 输入数据路径
output_dir = "shap_analysis_p"                     # 输出目录
target_column = "SO2 tolerance"                   # 目标变量列名
random_state = 42                                 # 随机种子

# 设置英文字体
plt.rcParams["font.family"] = ["Arial"]
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题


# ==================== 函数定义 ====================
def load_data_and_model():
    """加载数据和模型"""
    model = joblib.load(model_path)
    df = pd.read_excel(input_path)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
    return X[numeric_cols], y, model

def prepare_shap_data(X, y):
    """准备SHAP分析数据"""
    return X.sample(n=min(235, len(X)), random_state=random_state)

def plot_shap_summary(shap_values, X, output_dir):
    """绘制SHAP摘要图"""
    plt.figure(figsize=(14, 10), facecolor='white')
    
    # 计算合理的x轴范围
    max_shap = max(abs(shap_values.min()), abs(shap_values.max()))
    xlim = (-max_shap * 1.1, max_shap * 1.1)  # 稍微扩大范围
    
    # 适配新版SHAP
    if isinstance(shap_values, list):
        shap.summary_plot(shap_values[0], X, show=False, plot_size=None)
    else:
        shap.summary_plot(shap_values, X, show=False, plot_size=None)
    
    ax = plt.gca()
    ax.grid(False)
    ax.set_facecolor('white')
    ax.set_xlim(xlim)  # 设置对称的x轴范围
    
    # 设置标题样式
    plt.title("SHAP Value Distribution by Feature Importance",
              fontsize=26,
              pad=20,
              fontname='Arial',
              fontweight='bold')

    # ===== 坐标轴标签设置 =====
    ax.set_xlabel("SHAP Value",
                  fontsize=26,
                  fontname='Arial',
                  fontweight='bold',
                  labelpad=10)

    ax.set_ylabel("Feature",
                  fontsize=22,
                  fontname='Arial',
                  fontweight='bold',
                  labelpad=10)

    # ===== 刻度标签设置 =====
    ax.tick_params(axis='x',
                   which='both',
                   labelsize=22,
                   width=1.5,
                   labelfontfamily='Arial',
                   length=6,
                   direction='out')

    ax.tick_params(axis='y',
                   which='both',
                   labelsize=22,
                   width=1.5,
                   labelfontfamily='Arial',
                   length=6,
                   direction='out')

    # 设置坐标轴样式
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(3)
    
    # 保存图形
    plot_path = os.path.join(output_dir, "shap_summary.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()
    return plot_path

def plot_shap_bar(shap_values, X, output_dir):
    """绘制SHAP条形图"""
    plt.figure(figsize=(12, 8), facecolor='white')
    
    # 适配新版SHAP
    if isinstance(shap_values, list):
        shap.summary_plot(shap_values[0], X, plot_type="bar", show=False)
    else:
        shap.summary_plot(shap_values, X, plot_type="bar", show=False)
    
    ax = plt.gca()
    ax.grid(False)
    ax.set_facecolor('white')
    
    # 调整边距
    plt.subplots_adjust(left=0.3, right=0.95)
    
    # 设置坐标轴样式
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_linewidth(3)
    
    plot_path = os.path.join(output_dir, "shap_bar.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()
    return plot_path

def plot_shap_dependence(shap_values, X, output_dir):
    """绘制SHAP依赖图"""
    os.makedirs(os.path.join(output_dir, "dependence_plots"), exist_ok=True)
    plot_paths = []
    
    # 适配新版SHAP
    if isinstance(shap_values, list):
        shap_vals = shap_values[0]
    else:
        shap_vals = shap_values
    
    for feature in X.columns:
        plt.figure(figsize=(10, 6), facecolor='white')
        shap.dependence_plot(feature, shap_vals, X, show=False)
        
        ax = plt.gca()
        ax.grid(False)
        ax.set_facecolor('none')
        # 设置坐标轴样式
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.spines['bottom'].set_color('black')
        ax.spines['bottom'].set_linewidth(3)
        
        plot_path = os.path.join(output_dir, "dependence_plots", f"shap_dependence_{feature}.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
        plt.close()
        plot_paths.append(plot_path)
        
    return plot_paths
def plot_shap_waterfall(explainer, shap_values, X_sample, sample_index, output_dir):
    """绘制SHAP瀑布图"""
    os.makedirs(os.path.join(output_dir, "waterfall_plots"), exist_ok=True)

    plt.figure(figsize=(12, 10), facecolor='white')
    
    expected_value = explainer.expected_value
    if isinstance(expected_value, list):
        expected_value = expected_value[0]
    
    if isinstance(shap_values, list):
        shap_vals = shap_values[0]
    else:
        shap_vals = shap_values
    
    # 检查样本索引是否有效
    if sample_index >= len(X_sample):
        print(f"Warning: Sample index {sample_index} out of range, using last sample")
        sample_index = len(X_sample) - 1
    
    # 生成瀑布图
    shap.plots.waterfall(
        shap.Explanation(
            values=shap_vals[sample_index],
            base_values=expected_value,
            data=X_sample.iloc[sample_index],
            feature_names=X_sample.columns.tolist()
        ),
        show=False,
        max_display=10  # 限制显示的特征数量
    )

    ax = plt.gca()
    ax.grid(False)
    ax.set_facecolor('white')

    # 设置字体样式
    font_props = {
        'family': 'Arial',  # 字体类型
        'weight': 'bold',   # 字体加粗
        'size': 14          # 字体大小
    }
    
    # 设置坐标轴标签样式
    ax.set_xlabel(ax.get_xlabel(), fontdict=font_props)
    ax.set_ylabel(ax.get_ylabel(), fontdict=font_props)
    
    # 设置刻度标签样式
    ax.tick_params(axis='both', which='major', labelsize=14)
    for label in ax.get_xticklabels():
        label.set_fontweight('bold')
        label.set_fontname('Arial')
    for label in ax.get_yticklabels():
        label.set_fontweight('bold')
        label.set_fontname('Arial')
    
    # 设置标题样式（如果有）
    if ax.get_title():
        ax.set_title(ax.get_title(), fontdict={
            'fontsize': 14,
            'fontweight': 'bold',
            'fontname': 'Arial'
        })
    
    # 设置其他文本元素的样式（如特征名称等）
    for text in ax.texts:
        text.set_fontweight('bold')
        text.set_fontname('Arial')
        text.set_fontsize(14)

    # 调整边距
    plt.subplots_adjust(left=0.4, right=0.9)

    # 设置坐标轴样式
    ax.spines['top'].set_visible(False)
    ax.spines['top'].set_color('black')
    ax.spines['top'].set_linewidth(2)
    ax.spines['right'].set_visible(False)
    ax.spines['right'].set_color('black')
    ax.spines['right'].set_linewidth(2)
    ax.spines['left'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_visible(False)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(3.5)

    plot_path = os.path.join(output_dir, "waterfall_plots", f"shap_waterfall_{sample_index}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.close()
    return plot_path


def main():
    # 使用全局定义的输出目录，而不是重新定义
    global output_dir
    os.makedirs(output_dir, exist_ok=True)

    print("Loading data and model...")
    X, y, model = load_data_and_model()

    print("Preparing data for SHAP analysis...")
    X_shap = prepare_shap_data(X, y)

    print("Calculating SHAP values...")
    explainer = shap.TreeExplainer(model)
    
    # 适配新版SHAP API
    shap_values = explainer.shap_values(X_shap)
    if not isinstance(shap_values, list):
        # 确保shap_values是列表格式，兼容后续代码
        shap_values = [shap_values]

    print("Generating SHAP plots...")
    summary_path = plot_shap_summary(shap_values[0], X_shap, output_dir)
    bar_path = plot_shap_bar(shap_values[0], X_shap, output_dir)
    dependence_paths = plot_shap_dependence(shap_values, X_shap, output_dir)

    print("Generating Waterfall plots...")
    waterfall_paths = []
    sample_indices = [0, 5, min(233, len(X_shap)-1)]  # 确保索引不越界
    for i in sample_indices:
        try:
            path = plot_shap_waterfall(explainer, shap_values, X_shap, i, output_dir)
            waterfall_paths.append(path)
            print(f"Successfully generated waterfall plot for sample {i}")
        except Exception as e:
            print(f"Error generating waterfall plot for sample {i}: {str(e)}")
            import traceback
            traceback.print_exc()  # 打印完整的错误堆栈信息

    # 保存SHAP值
    shap_df = pd.DataFrame(shap_values[0], columns=X_shap.columns)
    shap_df.to_csv(os.path.join(output_dir, "shap_values.csv"), index=False)
    
    # 保存特征重要性排序
    importance_df = pd.DataFrame({
        'Feature': X_shap.columns,
        'Mean SHAP': abs(shap_df).mean().values
    })
    importance_df = importance_df.sort_values('Mean SHAP', ascending=False)
    importance_df.to_csv(os.path.join(output_dir, "feature_importance.csv"), index=False)

    print("\nSHAP analysis completed!")
    print(f"Results saved to: {os.path.abspath(output_dir)}")


if __name__ == "__main__":
    main()

Loading data and model...
Preparing data for SHAP analysis...
Calculating SHAP values...
Generating SHAP plots...
Generating Waterfall plots...
Successfully generated waterfall plot for sample 0
Successfully generated waterfall plot for sample 5
Successfully generated waterfall plot for sample 233

SHAP analysis completed!
Results saved to: C:\Users\miaoxinyi\NH3-SCR with SO2\shap_analysis_p


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [8]:
# -*- coding: utf-8 -*-
"""
优化版SHAP Force Plot脚本
功能：生成清晰的个体预测解释图，标注基准值和预测值
"""

import shap
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
import numpy as np
from matplotlib import rcParams

# ==================== 参数配置 ====================
MODEL_PATH = "xgboost_model_P/xgboost_model.pkl"  # 模型路径
DATA_PATH = "预测数据归一化后-20251120.xlsx"                      # 数据路径
OUTPUT_DIR = "shap_force_analysis_P"                # 输出目录
TARGET_COL = "SO2 tolerance"                     # 目标列名
SAMPLE_INDICES = [230, 231, 232, 233, 234, 235]                    # 指定分析样本索引
DPI = 300                                        # 输出分辨率

# 可视化参数
FONT_CONFIG = {
    'family': 'Times New Roman',
    'size': 22
}

COLOR_PALETTE = {
    'positive': '#ff0051',  # 正向影响(红色)
    'negative': '#008bfb',  # 负向影响(蓝色)
    'baseline': '#333333'   # 基准线颜色
}

# ==================== 主函数 ====================
def main():
    print("====== SHAP Force Plot 分析开始 ======")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # 设置全局样式
    rcParams.update({
        'font.family': FONT_CONFIG['family'],
        'font.size': FONT_CONFIG['size'],
        'figure.facecolor': 'white'
    })

    # 加载数据和模型
    print("\n🔍 加载数据和模型...")
    model = joblib.load(MODEL_PATH)
    df = pd.read_excel(DATA_PATH)
    X = df.drop(columns=[TARGET_COL]).select_dtypes(include=['number'])
    feature_names = X.columns.tolist()
    
    # 初始化解释器
    explainer = shap.TreeExplainer(model)
    
    print("\n📊 正在生成Force Plot...")
    for idx in SAMPLE_INDICES:
        try:
            sample = X.iloc[[idx]]
            shap_values = explainer.shap_values(sample)
            
            # 计算预测值
            base_value = explainer.expected_value
            predicted_value = base_value + np.sum(shap_values[0])
            
            # 创建图形
            plt.figure(figsize=(12, 8), facecolor='white')
            ax = plt.gca()
            
            # 绘制force plot
            force_plot = shap.force_plot(
                base_value=base_value,
                shap_values=shap_values,
                features=sample,
                feature_names=feature_names,
                show=False,
                matplotlib=True,
                plot_cmap=[COLOR_PALETTE['positive'], COLOR_PALETTE['negative']],
                text_rotation=45  # 特征标注旋转45度
            )
            
            # 标注基准值和预测值
            plt.axvline(x=base_value, color=COLOR_PALETTE['baseline'], linestyle='--', 
                        label=f'Base Value: {base_value:.3f}')
            plt.axvline(x=predicted_value, color='green', linestyle=':', 
                       label=f'Predicted Value: {predicted_value:.3f}')
            
            # 添加图例到右侧
            plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            
            # 调整布局
            plt.tight_layout()
            
            # 保存图形
            output_path = os.path.join(OUTPUT_DIR, f"force_plot_sample_{idx}.png")
            plt.savefig(output_path, dpi=DPI, bbox_inches='tight')
            plt.close()
            
            print(f"✅ 样本 {idx} 分析完成 -> {output_path}")
            
        except Exception as e:
            print(f"❌ 样本 {idx} 分析失败: {str(e)}")

    print("\n🎉 分析完成！结果保存在:", os.path.abspath(OUTPUT_DIR))

if __name__ == "__main__":
    main()

====== SHAP Force Plot 分析开始 ======

🔍 加载数据和模型...

📊 正在生成Force Plot...
✅ 样本 230 分析完成 -> shap_force_analysis_P\force_plot_sample_230.png
✅ 样本 231 分析完成 -> shap_force_analysis_P\force_plot_sample_231.png
✅ 样本 232 分析完成 -> shap_force_analysis_P\force_plot_sample_232.png
✅ 样本 233 分析完成 -> shap_force_analysis_P\force_plot_sample_233.png
✅ 样本 234 分析完成 -> shap_force_analysis_P\force_plot_sample_234.png
✅ 样本 235 分析完成 -> shap_force_analysis_P\force_plot_sample_235.png

🎉 分析完成！结果保存在: C:\Users\miaoxinyi\NH3-SCR with SO2\shap_force_analysis_P


In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import joblib
import os
import xgboost as xgb

# ==================== 参数配置 ====================
model_path = "xgboost_model_p/xgboost_model.pkl"  # 训练好的Booster模型
new_data_path = "归一化数据-20251120-SO2 tolerance prediciton.xlsx"                     # 待预测数据
output_path = "SO2 tolerance prediction_results.xlsx"             # 预测结果输出
target_column = "SO2 tolerance"                     # 目标变量名（新数据中可不存在）

# ==================== 函数定义 ====================
def load_and_prepare_new_data(new_data_path, model_features, train_median):
    """
    加载新数据，处理缺失值，并确保特征与模型一致
    """
    # 1. 加载新数据
    if new_data_path.endswith('.csv'):
        df_new = pd.read_csv(new_data_path)
    elif new_data_path.endswith('.xlsx'):
        df_new = pd.read_excel(new_data_path)
    else:
        raise ValueError("仅支持 .csv 或 .xlsx 格式")

    print(f"原始新数据形状: {df_new.shape}")

    # 2. 处理缺失值（用训练集的中位数填充）
    missing_mask = df_new.isnull()
    if missing_mask.any().any():
        print("\n检测到缺失值，用训练集中位数填充：")
        df_filled = df_new.fillna(train_median)
        # 打印填充信息
        filled_cols = missing_mask.sum()[missing_mask.sum() > 0].index
        for col in filled_cols:
            count = missing_mask[col].sum()
            print(f"  - {col}: {count} 个缺失值，填充值 = {train_median[col]:.4f}")
    else:
        print("未检测到缺失值")
        df_filled = df_new.copy()

    # 3. 确保特征与模型训练时一致
    missing_features = set(model_features) - set(df_filled.columns)
    if missing_features:
        raise ValueError(f"新数据缺少必需特征: {', '.join(missing_features)}")
    
    # 按模型特征顺序排列
    X_processed = df_filled[model_features].copy()
    print(f"\n预处理完成，特征矩阵形状: {X_processed.shape}")
    return df_new, X_processed

def main():
    # 1. 加载训练好的Booster模型
    print(f"正在加载模型: {model_path}")
    if not os.path.exists(model_path):
        print(f"错误: 模型文件 '{model_path}' 不存在")
        return
    model = joblib.load(model_path)
    print("模型加载成功")

    # 2. 获取模型训练时的特征名（关键修复：直接访问 feature_names）
    model_features = model.feature_names
    print(f"\n模型训练时使用的特征 ({len(model_features)} 个):")
    print(f"  {', '.join(model_features)}")

    # 3. 加载训练集数据，获取缺失值填充用的中位数（与训练时保持一致）
    train_data_path = "归一化数据-20251120.xlsx"
    if not os.path.exists(train_data_path):
        print(f"错误: 训练数据 '{train_data_path}' 不存在，无法获取填充策略")
        return
    
    print(f"\n从训练数据获取填充用的中位数...")
    df_train = pd.read_excel(train_data_path)
    # 只保留模型使用的特征，计算中位数
    train_features_df = df_train[model_features]
    train_median = train_features_df.median().to_dict()
    print("填充策略获取成功")

    # 4. 加载并预处理新数据
    try:
        df_original_new, X_new_processed = load_and_prepare_new_data(
            new_data_path, model_features, train_median
        )
    except Exception as e:
        print(f"\n预处理失败: {str(e)}")
        return

    # 5. 执行预测（Booster对象需要用DMatrix包装）
    print("\n正在预测...")
    dmatrix_new = xgb.DMatrix(X_new_processed)
    predictions = model.predict(dmatrix_new)
    print("预测完成")

    # 6. 保存结果
    print(f"\n保存结果至: {output_path}")
    df_results = df_original_new.copy()
    df_results[f"Predicted_{target_column}"] = predictions
    df_results.to_excel(output_path, index=False)
    print("结果保存成功！")

    # 7. 预览结果
    print("\n--- 预测结果预览 ---")
    preview_cols = list(df_results.columns[:min(5, len(df_results.columns)-1)]) + [f"Predicted_{target_column}"]
    print(df_results[preview_cols].head(10))

if __name__ == "__main__":
    main()

正在加载模型: xgboost_model_p/xgboost_model.pkl
模型加载成功

模型训练时使用的特征 (19 个):
  EN., I.E., I.R., M.P., T.C., Density, C.Tem., C.Tim., S.A., P.V., P.D., SO2, NH3, NO, O2, H2O, GHSV , P.Tim., P.Tem.

从训练数据获取填充用的中位数...
填充策略获取成功
原始新数据形状: (248, 19)
未检测到缺失值

预处理完成，特征矩阵形状: (248, 19)

正在预测...
预测完成

保存结果至: SO2 tolerance prediction_results.xlsx
结果保存成功！

--- 预测结果预览 ---
        EN.      I.E.      I.R.      M.P.      T.C.  Predicted_SO2 tolerance
0  0.767471  0.945662  0.615082  0.551861  0.972925                 0.272027
1  0.767471  0.945662  0.615082  0.551861  0.976339                 0.256551
2  0.754660  0.948052  0.576412  0.549170  0.997813                 0.334440
3  0.754660  0.948052  0.576412  0.549170  0.997813                 0.314221
4  0.754660  0.948052  0.576412  0.549170  0.997813                 0.303096
5  0.774003  0.946012  0.581822  0.551676  0.998050                 0.297058
6  0.774003  0.946012  0.581822  0.551676  0.998050                 0.272533
7  0.774003  0.946012  0.581822 